In [ ]:
import sys
sys.path.append("../input/cpythonlibrary/cpython-master/")
sys.path.append("../input/timmmaster/")
sys.path.append("../input/sam-optimizer/sam-main/")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import albumentations
import torch.nn as nn
import timm
import torch.nn.functional as F
import gc
import cv2
import warnings
from sam import SAM

from torch.utils.data import DataLoader
from PIL import Image
from tqdm import tqdm
from Lib import copy 

warnings.filterwarnings("ignore")

In [ ]:
class config:
    TRAINING_FILE = "../input/petfinder-pawpularity-score/train.csv"
    TRAINING_IMAGE_PATH = "../input/petfinder-pawpularity-score/train/"
    DEVICE = torch.device("cuda")
    TRAIN_BATCH_SIZE = 64
    VALID_BATCH_SIZE = 64
    EPOCHS = 5
    
class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
new_train = pd.read_csv(config.TRAINING_FILE)

In [ ]:
# create folds
from sklearn import model_selection

new_train["kfold"] = -1    
new_train = new_train.sample(frac=1).reset_index(drop=True)
y = new_train.Pawpularity.values
kf = model_selection.StratifiedKFold(n_splits=5)

for f, (t_, v_) in enumerate(kf.split(X = new_train, y = y)):
    new_train.loc[v_, 'kfold'] = f

In [ ]:
class petFinderDataset:
    def __init__(self, constant_func, dataframe, is_valid = 0):
        self.constant_func = constant_func
        self.dataframe = dataframe
        self.is_valid = is_valid
        if self.is_valid == 1: # transforms for validation images
            self.aug = albumentations.Compose([
            albumentations.RandomResizedCrop(224, 224),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            )], p=1.)

        else:       # transfoms for training images 
            self.aug = albumentations.Compose([
            albumentations.RandomResizedCrop(224, 224),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5)], p=1.)
    
    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, item):
        df = self.dataframe.iloc[item, :]

        # converting jpg format of images to numpy array
        img = np.array(Image.open(self.constant_func.TRAINING_IMAGE_PATH + df["Id"] + '.jpg')) 
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.aug(image = img)['image']
        img = np.transpose(img , (2,0,1)).astype(np.float32) # 2,0,1 because pytorch excepts image channel first then dimension of image
        
        return {
            'image': torch.tensor(img, dtype = torch.float),
            'tabular_data' : torch.tensor(df[1:-2], dtype = torch.float),
            'target' : torch.tensor(df['Pawpularity'], dtype = torch.float)
        }

In [ ]:
img = petFinderDataset(config, new_train)[71]['image']
plt.imshow(np.transpose(img.numpy(), (1,2,0)))

In [ ]:

class TimmEfficientNet_b0(nn.Module):
    def __init__(self):
        super(TimmEfficientNet_b0, self).__init__()
        self.model = timm.create_model("tf_efficientnet_b0_ns", pretrained=True, in_chans=3)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(128 + 12, 1)
        
    def forward(self, image ,tabular_data_inputs):
        x = self.model(image)
        x = self.dropout(x)
        x = torch.cat([x, tabular_data_inputs], dim=1)
        x = self.out(x)
        
        return x
    
model = TimmEfficientNet_b0()
model = model.to(config.DEVICE)

In [ ]:
base_optimizer = torch.optim.Adam
optimizer = SAM(model.parameters(), base_optimizer, lr = 1e-3 * 0.95)

scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
    )

def loss_fn(x, y):
    criterion = nn.MSELoss()
    loss = torch.sqrt(criterion(x, y))
    return loss

In [ ]:
def train_loop_fn(data_loader, model, optimizer, device, scheduler=None):
    running_loss = 0
    model.train()
    
    losses = AverageMeter()
    tqdm_ob = tqdm(data_loader, total = len(data_loader))
    
    for batch_index,dataset in enumerate(data_loader):
        image = dataset["image"]
        tabular_data = dataset["tabular_data"]
        target = dataset["target"]
        
        image = image.to(device, dtype=torch.float)
        tabular_data = tabular_data.to(device, dtype=torch.float)
        target = target.to(device, dtype=torch.float)

        # first forward-backward pass
        outputs = model(image, tabular_data)
        loss = loss_fn(outputs , target)
        loss.backward()
        optimizer.first_step(zero_grad=True)
        
        # second forward-backward pass
        outputs = model(image, tabular_data)
        loss = loss_fn(outputs , target)
        loss.backward()
        optimizer.second_step(zero_grad=True)
        
        scheduler.step()
        losses.update(loss.item(), image.size(0))
        tqdm_ob.set_postfix(loss = losses.avg)
            
        del image, tabular_data, target
        gc.collect()
        torch.cuda.empty_cache()
        
        running_loss += loss.item() 
    train_loss = running_loss/ (batch_index + 1)
    return train_loss

In [ ]:
def eval_loop_fn(data_loader, model, device):
    running_loss = 0
    model.eval()
    
    losses = AverageMeter()
    tqdm_ob = tqdm(data_loader, total = len(data_loader))
    
    for batch_index,dataset in enumerate(data_loader):
        image = dataset["image"]
        tabular_data = dataset["tabular_data"]
        target = dataset["target"]
        
        image = image.to(device, dtype=torch.float)
        tabular_data = tabular_data.to(device, dtype=torch.float)
        target = target.to(device, dtype=torch.float)

        outputs = model(image, tabular_data)
        loss = loss_fn(outputs , target)
        losses.update(loss.item(), image.size(0))
        tqdm_ob.set_postfix(loss = losses.avg)
            
        del image, tabular_data, target
        gc.collect()
        torch.cuda.empty_cache()
        
        running_loss += loss.item() 
    val_loss = running_loss/ (batch_index + 1)
    return val_loss

In [ ]:
def run():
    a_string = "*" * 20
    for i in range(5):
        print(a_string, " FOLD NUMBER ", i, a_string)
        df_train = new_train[new_train.kfold != i].reset_index(drop=True)
        df_valid = new_train[new_train.kfold == i].reset_index(drop=True)

        train_data = petFinderDataset(config, df_train)
        val_data = petFinderDataset(config, df_valid, is_valid = 1)

        train_data_loader = DataLoader(train_data,
                                num_workers=4,
                                batch_size=config.TRAIN_BATCH_SIZE,
                                shuffle=True,
                                drop_last=True)

        valid_data_loader = DataLoader(val_data,
                                num_workers=4,
                                batch_size=config.VALID_BATCH_SIZE,
                                shuffle=False,
                                drop_last=False)
        
        all_rmse = []
        for epoch in range(config.EPOCHS):
            print(f"Epoch --> {epoch+1} / {config.EPOCHS}")
            print(f"-------------------------------")
            train_rmse = train_loop_fn(train_data_loader, model, optimizer, config.DEVICE, scheduler)
            print(f"Training Root Mean Square Error = {train_rmse}")
            val_rmse = eval_loop_fn(valid_data_loader, model, config.DEVICE)
            print(f"Validation Root Mean Square Error = {val_rmse}")
            
            all_rmse.append(val_rmse)
        print('\n')
        
        if i < 1:
            best_RMSE = min(all_rmse)
            best_model = copy.deepcopy(model)
            all_rmse = []
        else:
            if min(all_rmse) < best_RMSE:
                best_RMSE = min(all_rmse)
                best_model = copy.deepcopy(model)
                all_rmse = []
                
    torch.save(best_model,'./second_timm_efficientnet_b0_ns_model.bin')
    print()
    print(f"The lowest RMSE score that we got across all the folds is : {best_RMSE}")
    
    return best_model
                
if __name__ == "__main__":
    run()